In [2]:
import re
import pandas as pd
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from pandas.core.frame import DataFrame
import json
from datetime import datetime, timezone
from text_preprocessor import TextPreprocessor

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jereluck20/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jereluck20/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jereluck20/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jereluck20/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jereluck20/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jereluck20/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from utils import read_comments_s3
data = read_comments_s3()
df = pd.DataFrame(json.loads(data))
df

,post,author,author_comment_karma,author_link_karma,author_is_mod,author_is_gold,comment_id,body,score,date
0,Rate My Portfolio - r/Stocks Quarterly Thread ...,iseeyiy,9464,1,False,False,hmvieec,100% TSLA,34,1638408827.0
1,Rate My Portfolio - r/Stocks Quarterly Thread ...,Randomtv126,1630,203,False,False,hnf8rfc,I have been investing for about four months no...,29,1638766423.0
2,Rate My Portfolio - r/Stocks Quarterly Thread ...,aznkor,4635,352,False,False,ho3huf9,100% QQQ\n\n1. Sell daily OTM calls\n2. Reinve...,19,1639208956.0
3,Rate My Portfolio - r/Stocks Quarterly Thread ...,alexvndvr,638,612,False,False,hmsfnyy,80% VTSAX\n\n20% VTIAX,18,1638362590.0
4,Rate My Portfolio - r/Stocks Quarterly Thread ...,PanPirat,21460,4459,True,False,hn1ot2t,I just went through some adjustments in my por...,15,1638520872.0
...,...,...,...,...,...,...,...,...,...,...
294,Too many of you have never experienced a stock...,Raleigh_Noles,15300,4954,False,False,hqxofeq,"A big difference between buying the dip, and l...",55,1641132656.0
295,Too many of you have never experienced a stock...,GhostSierra117,113912,12285,True,False,hqx38n0,That's the correct answer.\n\nA lot of states ...,55,1641117034.0
296,Too many of you have never experienced a stock...,SnydersCordBish,19348,3097,False,False,hqyftyw,6 months salary cash in the bank. Everyone sho...,14,1641144691.0
297,Too many of you have never experienced a stock...,Outrageous-Cycle-841,3511,13,False,False,hqxt2d6,Agreed. Good outcome but bad process.,25,1641135081.0


In [26]:
def copy_df(df):
   return df.copy()

def drop_unpopular_comments(df):
    MIN_SCORE = 10
    return df[df['score'].astype(int) >= MIN_SCORE]

def drop_irrelevant_posts(df):
    excluded_submission_keywords = ['Thread', 'Daily Discussion', 'Rate My Portfolio']
    pattern = '|'.join(excluded_submission_keywords)
    return df[~df['post'].str.contains(pattern)]

def convert_time_zone(date):
    utc_datetime = datetime.utcfromtimestamp(float(date))
    local_datetime = utc_datetime.replace(tzinfo=timezone.utc).astimezone(tz=None)
    local_datetime = local_datetime.strftime('%Y-%m-%d %H:%M:%S')
    return local_datetime

def convert_date_time_zone(df):
    df['date'] = df['date'].apply(convert_time_zone)
    return df

def upper_case_is_mod(df):
    df['author_is_mod'] = df['author_is_mod'].apply(lambda x: str(x).upper())
    return df

def upper_case_is_gold(df):
    df['author_is_gold'] = df['author_is_gold'].apply(lambda x: str(x).upper())
    return df

def preprocess_comment_body(df):
    text_preprocessor = TextPreprocessor()
    df['body'] = df['body'].apply(text_preprocessor.preprocess_text)
    return df


In [25]:
df_copy = copy_df(df)
df_copy .pipe(drop_unpopular_comments) \
        .pipe(drop_irrelevant_posts) \
        .pipe(convert_date_time_zone) \
        .pipe(upper_case_is_mod) \
        .pipe(upper_case_is_gold) \
        .pipe(preprocess_comment_body) 

,post,author,author_comment_karma,author_link_karma,author_is_mod,author_is_gold,comment_id,body,score,date
32,Tesla beats Wall Street expectations again,Kookiano,1692,1,FALSE,FALSE,hqydy5u,number car wall street expect,123,2022-01-02 12:19:34
33,Tesla beats Wall Street expectations again,juaggo_,27640,19602,TRUE,FALSE,hqydin3,middl chip shortag insan execut tesla team,546,2022-01-02 12:16:53
34,Tesla beats Wall Street expectations again,garlicroastedpotato,425949,4746,FALSE,FALSE,hqz3hn2,let see b revenu year valuat next year worth,142,2022-01-02 14:56:32
35,Tesla beats Wall Street expectations again,glr55,617,3,FALSE,FALSE,hqygsuf,person invest compani deni one success public ...,158,2022-01-02 12:37:32
37,Tesla beats Wall Street expectations again,esb219,6786,60,FALSE,FALSE,hqyaa0a,underli financi start catch valuat overvalu st...,154,2022-01-02 11:56:07
38,Tesla beats Wall Street expectations again,kenypowa,19050,15335,FALSE,FALSE,hqydrot,w consensu k massiv beat tesla actual annual p...,148,2022-01-02 12:18:26
46,Tesla beats Wall Street expectations again,DyIanLL,11669,6,FALSE,FALSE,hqy8v0k,r stock post bare get upvot comment bc posit t...,140,2022-01-02 11:46:51
53,Tesla beats Wall Street expectations again,diamond_dav,1495,482,FALSE,FALSE,hqyoq7s,agre set asid commentari musk stock price reco...,165,2022-01-02 13:27:11
58,Tesla beats Wall Street expectations again,BrexitBabyYeah,5597,650,FALSE,FALSE,hqydd4j,p e keep come,115,2022-01-02 12:15:58
93,Tesla beats Wall Street expectations again,MDrag1992,15143,11395,FALSE,FALSE,hqzmiis,work low end product tesla millionair get grea...,118,2022-01-02 16:57:10


In [ ]:
# NLP processing